# Setup

In [3]:
#setup path
path = 'C:/Users/mark/Documents/GitHub/honours/dev/package/'
import sys; sys.path.append(path)

#standard imports
import numpy as np
import pandas as pd
import datetime
from scipy import stats
from collections import OrderedDict

#configure display modes
from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
import matplotlib.dates as md
from matplotlib import rcParams
import seaborn as sns

%matplotlib inline
rcParams['figure.figsize'] = 20,6
sns.set_style("whitegrid")

#load data first time from database
import analysis.dataPreparer as dp

input_path = path+"analysis/input/test - Copy.db"
flatten_config = path+"analysis/config/master_feature.sql"
raw_flat_file = path+"analysis/output/temp/raw.csv"
clean_flat_file = path+"analysis/output/temp/clean.csv"
output_path = path+"analysis/output/temp/output.db"

dp.flatten_file(input_path, flatten_config, raw_flat_file, "temp")
dp.clean_file(raw_flat_file, clean_flat_file)
dp.load_file(output_path, clean_flat_file, "temp")
df = dp.export_dataframe(output_path, "temp")
df_backup = df.copy()

logManager - INFO - flatten_file | Started
logManager - INFO - flatten | Started
logManager - ERROR - flatten | Failed | 89.98
Traceback (most recent call last):
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\logManager.py", line 58, in wrapper
    try: result = f(*args, **kwargs)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\analysis\dataPreparer.py", line 257, in flatten
    sm.execute_script(database_file, script_file)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\sqlManager.py", line 74, in execute_script
    conn.executescript(script)
sqlite3.OperationalError: no such column: event_relationships.entity_uuid


logManager - ERROR - flatten | Failed | 89.98
Traceback (most recent call last):
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\logManager.py", line 58, in wrapper
    try: result = f(*args, **kwargs)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\analysis\dataPreparer.py", line 257, in flatten
    sm.execute_script(database_file, script_file)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\sqlManager.py", line 74, in execute_script
    conn.executescript(script)
sqlite3.OperationalError: no such column: event_relationships.entity_uuid


dbLoader - INFO - C:/Users/mark/Documents/GitHub/honours/dev/package/analysis/output/temp/raw.csv | Already deleted
dbLoader - INFO - temp.csv | Export started
logManager - ERROR - flatten_file | Failed | 90.01
Traceback (most recent call last):
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\logManager.py", line 58, in wrapper
    try: result = f(*args, **kwargs)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\analysis\dataPreparer.py", line 262, in flatten_file
    db.export_file(database_file, export_file, file_name)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\dbLoader.py", line 94, in export_file
    try: odo.odo(table_uri, export_file)
  File "C:\Anaconda3\lib\site-packages\odo\odo.py", line 91, in odo
    return into(target, source, **kwargs)
  File "C:\Anaconda3\lib\site-packages\multipledispatch\dispatcher.py", line 164, in __call__
    return func(*args, **kwargs)
  File "C:\Anaconda3\lib\site-packages\odo\into.py", line 43, in wrapped
    

logManager - ERROR - flatten_file | Failed | 90.01
Traceback (most recent call last):
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\logManager.py", line 58, in wrapper
    try: result = f(*args, **kwargs)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\analysis\dataPreparer.py", line 262, in flatten_file
    db.export_file(database_file, export_file, file_name)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\dbLoader.py", line 94, in export_file
    try: odo.odo(table_uri, export_file)
  File "C:\Anaconda3\lib\site-packages\odo\odo.py", line 91, in odo
    return into(target, source, **kwargs)
  File "C:\Anaconda3\lib\site-packages\multipledispatch\dispatcher.py", line 164, in __call__
    return func(*args, **kwargs)
  File "C:\Anaconda3\lib\site-packages\odo\into.py", line 43, in wrapped
    return f(*args, **kwargs)
  File "C:\Anaconda3\lib\site-packages\odo\into.py", line 149, in into_string_string
    return into(a, resource(b, **kwargs), **kwarg

logManager - INFO - clean_file | Started
logManager - INFO - read | Started
logManager - ERROR - read | Failed | 0.00
Traceback (most recent call last):
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\logManager.py", line 58, in wrapper
    try: result = f(*args, **kwargs)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\analysis\dataPreparer.py", line 53, in read
    if nrows is None: df = pd.read_csv(file,low_memory=False)
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 646, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 389, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 730, in __init__
    self._make_engine(self.engine)
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 923, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "C:\Anaconda3\

...

logManager - INFO - load_file | Started
logManager - ERROR - load_file | Failed | 0.00
Traceback (most recent call last):
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\logManager.py", line 58, in wrapper
    try: result = f(*args, **kwargs)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\analysis\dataPreparer.py", line 273, in load_file
    else: df = pd.read_csv(clean_file, encoding="latin1")
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 646, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 389, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 730, in __init__
    self._make_engine(self.engine)
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 923, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "C:\Anaconda3\lib\site-packages\pandas\io\p

logManager - ERROR - load_file | Failed | 0.00
Traceback (most recent call last):
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\logManager.py", line 58, in wrapper
    try: result = f(*args, **kwargs)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\analysis\dataPreparer.py", line 273, in load_file
    else: df = pd.read_csv(clean_file, encoding="latin1")
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 646, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 389, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 730, in __init__
    self._make_engine(self.engine)
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 923, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "C:\Anaconda3\lib\site-packages\pandas\io\parsers.py", line 1390, in __init__
    s

logManager - INFO - export_dataframe | Started
logManager - ERROR - export_dataframe | Failed | 0.01
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\pandas\io\sql.py", line 257, in read_sql_table
    meta.reflect(only=[table_name], views=True)
  File "C:\Anaconda3\lib\site-packages\sqlalchemy\sql\schema.py", line 3650, in reflect
    (bind.engine.url, s, ', '.join(missing)))
sqlalchemy.exc.InvalidRequestError: Could not reflect: requested table(s) not available in sqlite:///C:/Users/mark/Documents/GitHub/honours/dev/package/analysis/output/temp/output.db: (temp)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\logManager.py", line 58, in wrapper
    try: result = f(*args, **kwargs)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\analysis\dataPreparer.py", line 287, in export_dataframe
    else: df = pd.read_sql_table(table, conn)
  File

logManager - ERROR - export_dataframe | Failed | 0.01
Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\pandas\io\sql.py", line 257, in read_sql_table
    meta.reflect(only=[table_name], views=True)
  File "C:\Anaconda3\lib\site-packages\sqlalchemy\sql\schema.py", line 3650, in reflect
    (bind.engine.url, s, ', '.join(missing)))
sqlalchemy.exc.InvalidRequestError: Could not reflect: requested table(s) not available in sqlite:///C:/Users/mark/Documents/GitHub/honours/dev/package/analysis/output/temp/output.db: (temp)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\logManager.py", line 58, in wrapper
    try: result = f(*args, **kwargs)
  File "C:/Users/mark/Documents/GitHub/honours/dev/package\analysis\dataPreparer.py", line 287, in export_dataframe
    else: df = pd.read_sql_table(table, conn)
  File "C:\Anaconda3\lib\site-packages\pandas\io\sql.

AttributeError: 'NoneType' object has no attribute 'copy'

In [5]:
#reload data from memory
df = df_backup

NameError: name 'df_backup' is not defined

In [ ]:
#create label
label = "outcome_exit_bool"
y = df[label]

#create features
df = df.select_dtypes(['number'])
drops = [col for col in list(df) if col.startswith(("key","from","outcome","index"))]
X = df.drop(drops, axis=1)
features = list(X)

In [ ]:
#Create lifecycle stages - 2013
df["keys_company_stage_seed"] = df["confidence_validation_funding_round_codes_list_seed"]
df["keys_company_stage_series-a"] = df["confidence_validation_funding_round_codes_list_a"]
df["keys_company_stage_series-b"] = df["confidence_validation_funding_round_codes_list_b"]
df["keys_company_stage_series-c+"] = df["confidence_validation_funding_round_types_list_series-c+"]
df["keys_company_stage_ipo"] = df["confidence_performance_other_ipo_bool"]
df["keys_company_stage_acquired"] = df["confidence_performance_other_acquired_bool"]
df["keys_company_stage_closed"] = df["confidence_performance_other_closed_bool"]

def label_stage(row):
    if row["keys_company_stage_closed"] >= 1: stage = "Closed"
    elif row["keys_company_stage_acquired"] >= 1: stage = "Acquired"
    elif row["keys_company_stage_ipo"] >= 1: stage = "IPO"
    elif row["keys_company_stage_series-c+"] >= 1: stage = "Series C+"
    elif row["keys_company_stage_series-b"] >= 1: stage = "Series B"
    elif row["keys_company_stage_series-a"] >= 1: stage = "Series A"
    elif row["keys_company_stage_seed"] >= 1: stage = "Seed"
    elif row["confidence_validation_funding_rounds_number"] == 0: stage = "None"
    else: stage = "Other"
    return stage

df["keys_company_stage"] = df.apply(lambda row: label_stage(row), axis=1)
company_stage = df["keys_company_stage"]

#Decompose Other and None stages
from sklearn.mixture import GaussianMixture
mix = GaussianMixture(n_components=2)

def label_stage_other(row, stage):
    if row["keys_company_stage_"+stage.lower()+"_young"] == 1: val = stage + "- Young"
    elif row["keys_company_stage_"+stage.lower()+"_young"] == 0: val = stage + "- Mature"
    else: val = row["keys_company_stage"]
    return val

founded_date = df["confidence_context_broader_founded_date"]

for stage in ["Other", "None"]:
    founded_date_other = founded_date.loc[df['keys_company_stage'] == stage].dropna()
    founded_date_other_stacked = np.vstack(founded_date_other)
    mix.fit(founded_date_other_stacked)
    pred = mix.predict(founded_date_other_stacked)
    max_index = list(mix.means_).index(max(mix.means_))
    if max_index == 0: pred = [0 if x==1 else 1 for x in pred]
    df["keys_company_stage_"+stage.lower()+"_young"] = pd.Series(pred, index=founded_date_other.index)
    df["keys_company_stage"] = df.apply(lambda row: label_stage_other(row, stage), axis=1)
    df["keys_company_stage_"+stage.lower()+"_mature"] = df.apply(lambda row: 1 if row["keys_company_stage"] == stage + "- Mature" else 0, axis=1)
    
df_backup = df.copy()

In [ ]:
#reload data from memory
df = df_backup

# 4. Evaluation

## 4.1 Data Collection

Table 4.1 Companies grouped by lifecycle stage.

In [3]:
#TODO - FIX Company Stage

df["survival_date"] = min(df["confidence_context_broader_closing_date"], df["keys_updated_at_date"])
df["keys_company_age"] = (df["survival_date"] - df["confidence_context_broader_founded_date"]) / 60 / 60 / 24 / 365.25

factors = dict(
    Included = ["Pre-Seed", "Angel", "Seed", "Series A", "Series B", "Series C+"],
    Excluded = ["Closed", "Acquired", "IPO", "Non-Startup"],
    Other = ["None", "Other"])

f = {v:k for k,v in {v:k for k,v in factors.items()}}

df["keys_company_stage_group"] = df["keys_company_stage"].map(factors)


groups = df[label, "keys_company_stage_group", "keys_company_stage"]
columns = ["keys_company_stage", "keys_company_age", "confidence_validation_funding_total_value_number", "confidence_validation_funding_rounds_number"]
colnames = ["Observations", "Age (Years)", "Funding Raised (USD)", "Funding Rounds"]
colfuncs = [np.sum, [np.median, stats.iqr], [np.median, stats.iqr], [np.median, stats.iqr]]

namefunc = OrderedDict(zip(columns, colnames))
aggfunc = OrderedDict(zip(columns, colfuncs))

tab = df[columns].groupby([groups]).describe()
#tab = tab.agg(aggfunc)
tab = tab.rename(columns=namefunc)

NameError: name 'df' is not defined

Figure 4.1 Company ages by lifecycle stage (KDE plot).

## 4.2 Feature Extraction

In [ ]:
#make constraints
df = df.loc[df['keys_company_stage_group'] in ["Included", "Other"]]

#update X and y
X = df[features]
y = df[label]

### 4.2.1 Missing Values

Figure 4.2 Missing features per observation (histogram).

In [ ]:
missing_by_row = X.isnull().sum(axis=1)
g = sns.distplot(missing_by_row, bins=20)
g.axes.set_xlim(0,len(list(df)))

Figure 4.3 Missing observations per feature (histogram).

In [ ]:
missing_by_col =  X.isnull().sum(axis=0)
g = sns.distplot(missing_by_col,bins=20)
g.axes.set_xlim(0,len(df.index))

### 4.2.2 Feature Components

Figure 4.4 Eigenvalues extracted from PCA model (line plot).

In [ ]:
#Principal Components Analysis (PCA)
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

def screeplot(pca, X_std):
    y = np.std(pca.transform(X_std), axis=0)**2
    x = np.arange(len(y)) + 1
    plt.plot(x, y)
    plt.ylabel("Eigenvalue")
    plt.xlabel("Component Number")
    plt.axhline(y=1)
    return y

X_std = pd.DataFrame(scale(X), index=X.index, columns=X.columns)
pca = PCA().fit(X_std)
eigenvalues = screeplot(pca, X_std)
retained = np.where(eigenvalues >= 1)

Figure 4.5 Component loadings on each factor from conceptual framework (matrix).

In [ ]:
from collections import defaultdict

groups = [x.split("_")[2]) for x in features]

loadings_matrix = []
for x in retained:
    loadings = pca.components_[x]
    grp_loadings = defaultdict(int)
    for group, loading in zip(groups, loadings):
        grp_loadings[group] += loading
    loadings_matrix.append(grp_loadings)
print(loadings_matrix)

Figure 4.6 Inter-correlations of each factor from conceptual framework (matrix).

In [ ]:
def sum_group(row):
    col_groups = defaultdict(int)
    for group, val in zip(col_groups, list(row)):
        col_groups[group] += val
    row = pd.Series(col_groups)
    return row

X_grp = X_std.apply(lambda row: sum_group(row), axis=1)

sns.heatmap(X_grp.corr(method="spearman"), square=True)

## 4.3 Classification Algorithms

Table 4.2 Classification accuracy metrics for each algorithm.

Figure 4.7 Receiver operating characteristics for each algorithm (line plot).

## 4.4 Experimentation

### 4.4.1 Practical 

Figure 4.8 AUC ROC for different training set sizes (line plot).

Table 4.3 Time profiling by system component.

### 4.4.2 Robust

Table 4.4 Comparison of 2013 slice from 2016 dataset with 2013 dataset.

Table 4.5 Features ranked by importance for different time slices.

Figure 4.9 ROC curves for models trained on different time slices (line plot).

### 4.4.3 Predictive

Table 4.6 Features ranked by importance for different prediction windows.

Figure 4.10 ROC curves for models trained on different prediction windows (line plot).

Figure 4.11 Change in company stage over different prediction windows (matrix).

Figure 4.12 ROC curves for target companies at different lifecycle stages (line plot).

Figure 4.13 AUC ROC for models by lifecyle stage and prediction window (matrix)

Table 4.7 Three example company profiles and their predictions.

# C. Conceptual Framework

Table C.1 List of included features.

# D. Additional Classifier Results

Table D.1 Hyperparameter gridsearch for each algorithm.

Table D.2 Optimal Logistic Regression model.

Figure D.1 Optimal Decision Tree model (tree).